In [ ]:
# Instalar todo en una línea
!pip install pandas numpy sqlalchemy openpyxl matplotlib seaborn

# O instalarlas por separado:
!pip install pandas
!pip install numpy
!pip install sqlalchemy
!pip install openpyxl
!pip install matplotlib
!pip install seaborn

In [1]:
# Librerías básicas esenciales
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Para SQL y base de datos
from sqlalchemy import create_engine
import sqlite3  # Viene con Python, no necesita instalación

# Para Excel (OPENPYXL es el más recomendado)
import openpyxl

# Opcional: para visualizaciones (si quieres mostrar gráficos)
import matplotlib.pyplot as plt
import seaborn as sns

# Para manejo de archivos y sistema
import os

In [2]:
def cargar_y_limpiar_excel(excel_path):
    """Carga todas las hojas de Excel y maneja filas en blanco CORREGIDO"""
    excel_file = pd.ExcelFile(excel_path)
    dataframes_limpios = {}

    for sheet_name in excel_file.sheet_names:
        print(f" Procesando hoja: {sheet_name}")

        # Cargar datos crudos sin asumir headers
        df_raw = pd.read_excel(excel_path, sheet_name=sheet_name, header=None)

        # Eliminar filas completamente vacías al inicio
        df_clean = df_raw.dropna(how='all')

        if df_clean.empty:
            print(f"    Hoja {sheet_name} está vacía")
            continue

        # Buscar la fila que contiene los headers reales
        header_row_idx = None
        for idx in range(min(3, len(df_clean))):  # Revisar solo las primeras 3 filas
            fila_actual = df_clean.iloc[idx]
            # Verificar si esta fila tiene valores que parecen nombres de columnas
            valores_fila = [str(val).lower() for val in fila_actual if pd.notna(val)]

            # Palabras clave que esperamos en los headers
            palabras_clave = ['sku', 'orden', 'producto', 'cantidad', 'precio', 'fecha',
                            'cliente', 'categoria', 'stock', 'proveedor', 'ventas', 'inventario']

            if any(any(palabra in valor for palabra in palabras_clave) for valor in valores_fila):
                header_row_idx = idx
                print(f"    Header encontrado en fila {idx+1}")
                break

        if header_row_idx is not None:
            # Usar esa fila como header y tomar los datos desde la siguiente fila
            df_clean.columns = df_clean.iloc[header_row_idx]
            df_clean = df_clean.iloc[header_row_idx + 1:].reset_index(drop=True)
        else:
            # Si no encontramos headers, usar los primeros como default
            print(f"    No se encontraron headers claros, usando primera fila")
            df_clean.columns = df_clean.iloc[0]
            df_clean = df_clean.iloc[1:].reset_index(drop=True)

        # Eliminar cualquier fila que quede completamente vacía
        df_clean = df_clean.dropna(how='all')

        # Renombrar columnas para consistencia (eliminar espacios extra, etc.)
        df_clean.columns = [str(col).strip() if pd.notna(col) else f"col_{i}"
                          for i, col in enumerate(df_clean.columns)]

        dataframes_limpios[sheet_name] = df_clean
        print(f"    Registros limpios: {len(df_clean)}")
        print(f"    Columnas: {list(df_clean.columns)}")

    return dataframes_limpios

# Probar la función de limpieza corregida
print(" Limpiando datos del Excel...")
dataframes = cargar_y_limpiar_excel('datos_empresa_completo.xlsx')

 Limpiando datos del Excel...
 Procesando hoja: ventas_online_enero
    Header encontrado en fila 1
    Registros limpios: 1000
    Columnas: ['orden_id', 'cliente_id', 'producto_online', 'cantidad_online', 'precio_unitario_online', 'fecha_compra_online', 'canal_online', 'rating_online']
 Procesando hoja: ventas_online_febrero
    Header encontrado en fila 1
    Registros limpios: 1000
    Columnas: ['orden_id', 'cliente_id', 'producto_online', 'cantidad_online', 'precio_unitario_online', 'fecha_compra_online', 'canal_online', 'rating_online']
 Procesando hoja: inventario_almacen_norte
    Header encontrado en fila 1
    Registros limpios: 1000
    Columnas: ['sku_almacen', 'producto_almacen', 'categoria_almacen', 'stock_actual_almacen', 'stock_minimo_almacen', 'proveedor_almacen', 'costo_unitario_almacen', 'ubicacion_almacen']
 Procesando hoja: ventas_tienda_marzo
    Header encontrado en fila 1
    Registros limpios: 1000
    Columnas: ['orden_id', 'cliente_id', 'producto_tienda', 'c

In [3]:
def cargar_excel_inteligente(excel_path, max_filas_busqueda=15):
    """Versión que maneja mejor headers en cualquier posición"""

    excel_file = pd.ExcelFile(excel_path)
    dataframes_limpios = {}

    for sheet_name in excel_file.sheet_names:
        print(f"\n Procesando: '{sheet_name}'")

        # Cargar TODAS las filas
        df_raw = pd.read_excel(excel_path, sheet_name=sheet_name, header=None)

        if df_raw.empty:
            print(f"    Hoja vacía")
            continue

        # Encontrar la primera fila no vacía
        primera_fila_no_vacia = None
        for idx in range(len(df_raw)):
            if not df_raw.iloc[idx].isna().all():
                primera_fila_no_vacia = idx
                break

        if primera_fila_no_vacia is None:
            print(f"    Todas las filas están vacías")
            continue

        print(f"    Primera fila con datos: {primera_fila_no_vacia + 1}")

        # Buscar header desde la primera fila no vacía
        header_row_idx = None
        fila_inicio_busqueda = primera_fila_no_vacia
        fila_fin_busqueda = min(primera_fila_no_vacia + max_filas_busqueda, len(df_raw))

        for idx in range(fila_inicio_busqueda, fila_fin_busqueda):
            fila_actual = df_raw.iloc[idx]
            valores_no_nulos = [str(val).lower() for val in fila_actual if pd.notna(val)]

            if not valores_no_nulos:
                continue
                #SKU --> sku
            # Palabras clave ampliadas
            palabras_clave = [
                'sku', 'orden', 'producto', 'cantidad', 'precio', 'fecha',
                'cliente', 'categoria', 'stock', 'proveedor', 'ventas', 'inventario',
                'id', 'nombre', 'descripcion', 'total', 'importe', 'monto', 'valor'
            ]

            # Verificar si parece un header
            coincidencias = sum(1 for valor in valores_no_nulos
                              for palabra in palabras_clave if palabra in valor)

            if coincidencias >= 2:  # Si hay al menos 2 coincidencias
                header_row_idx = idx
                print(f"    Header probable en fila {idx + 1} ({coincidencias} coincidencias)")
                break

        # Procesar según lo encontrado
        if header_row_idx is not None:
            df_clean = df_raw.iloc[header_row_idx + 1:].reset_index(drop=True)
            df_clean.columns = df_raw.iloc[header_row_idx]
            print(f"    Datos desde fila {header_row_idx + 2}")
        else:
            # Estrategia de fallback
            print(f"    Header no identificado claramente")
            df_clean = df_raw.iloc[primera_fila_no_vacia:].reset_index(drop=True)
            df_clean.columns = [f'col_{i}' for i in range(len(df_clean.columns))]

        # Limpieza final
        df_clean = df_clean.dropna(how='all')
        df_clean.columns = [str(col).strip().replace(' ', '_').lower()
                          if pd.notna(col) else f'col_{i}'
                          for i, col in enumerate(df_clean.columns)]

        dataframes_limpios[sheet_name] = df_clean
        print(f"    {len(df_clean)} registros, {len(df_clean.columns)} columnas")

    return dataframes_limpios

# Probar la versión más robusta
print(" CARGA INTELIGENTE DE EXCEL")
dataframes = cargar_excel_inteligente('datos_empresa_completo.xlsx')

 CARGA INTELIGENTE DE EXCEL

 Procesando: 'ventas_online_enero'
    Primera fila con datos: 1
    Header probable en fila 1 (9 coincidencias)
    Datos desde fila 2
    1000 registros, 8 columnas

 Procesando: 'ventas_online_febrero'
    Primera fila con datos: 1
    Header probable en fila 1 (9 coincidencias)
    Datos desde fila 2
    1000 registros, 8 columnas

 Procesando: 'inventario_almacen_norte'
    Primera fila con datos: 1
    Header probable en fila 1 (6 coincidencias)
    Datos desde fila 2
    1000 registros, 8 columnas

 Procesando: 'ventas_tienda_marzo'
    Primera fila con datos: 1
    Header probable en fila 1 (9 coincidencias)
    Datos desde fila 2
    1000 registros, 8 columnas

 Procesando: 'inventario_almacen_central'
    Primera fila con datos: 1
    Header probable en fila 1 (6 coincidencias)
    Datos desde fila 2
    1000 registros, 8 columnas

 Procesando: 'Hoja1'
    Hoja vacía

 Procesando: 'inventario_bodega_sur'
    Primera fila con datos: 2
    Header pr

In [11]:
def cargar_a_sql_agrupado(dataframes_dict, db_engine):
    """Carga los dataframes a SQL agrupando por tipo - VERSIÓN CORREGIDA"""

    # Primero, crear las tablas con estructura definida
    tablas_creadas = set()

    #CONTADOR PARA LOS REGISTROS
    contador_registros = {}

    for sheet_name, df in dataframes_dict.items():
        if df.empty:
            continue

        # Determinar el tipo de tabla basado en el nombre de la hoja
        if 'ventas' in sheet_name.lower():
            if 'online' in sheet_name.lower():
                table_name = 'ventas_online'
                # Columnas esperadas para ventas_online
                expected_columns = ['orden_id', 'cliente_id', 'producto_online', 'cantidad_online',
                                  'precio_unitario_online', 'fecha_compra_online', 'canal_online', 'rating_online']
            else:
                table_name = 'ventas_tienda'
                # Columnas esperadas para ventas_tienda
                expected_columns = ['orden_id', 'cliente_id', 'producto_tienda', 'cantidad_tienda',
                                  'precio_unitario_tienda', 'fecha_compra_tienda', 'sucursal_tienda', 'vendedor_tienda']
        elif 'inventario' in sheet_name.lower():
            if 'almacen' in sheet_name.lower():
                table_name = 'inventario_almacen'
                # Columnas esperadas para inventario_almacen
                expected_columns = ['sku_almacen', 'producto_almacen', 'categoria_almacen', 'stock_actual_almacen',
                                  'stock_minimo_almacen', 'proveedor_almacen', 'costo_unitario_almacen', 'ubicacion_almacen']
            else:
                table_name = 'inventario_bodega'
                # Columnas esperadas para inventario_bodega
                expected_columns = ['sku_bodega', 'producto_bodega', 'categoria_bodega', 'stock_actual_bodega',
                                  'stock_minimo_bodega', 'proveedor_bodega', 'costo_unitario_bodega', 'ubicacion_bodega']
        else:
            table_name = sheet_name.lower().replace(' ', '_')
            expected_columns = list(df.columns)

        # Agregar columna para identificar la fuente
        df['fuente_archivo'] = sheet_name

        if table_name not in contador_registros:
          contador_registros[table_name] = 0
        contador_registros[table_name] += len(df)

        # Si es la primera vez que vemos esta tabla, crear con replace
        if table_name not in tablas_creadas:
            df.to_sql(table_name, db_engine, index=False, if_exists='replace')
            tablas_creadas.add(table_name)
            print(f" Tabla '{table_name}' creada con {len(df)} registros")
        else:
            # Si ya existe, hacer append
            df.to_sql(table_name, db_engine, index=False, if_exists='append')
            print(f" Agregados {len(df)} registros a tabla: {table_name}")

    total_tabla=0
    for tabla, cantidad in contador_registros.items():
      print(f"{tabla}: {cantidad} registros")
      total_tabla += cantidad

    print(f"Total general : {total_tabla} registros")
    print(f"Total de tablas : {len(contador_registros)} tablas")


# Reiniciar la base de datos para empezar limpio
import os
if os.path.exists('analisis_empresa.db'):
    os.remove('analisis_empresa.db')

engine = create_engine('sqlite:///analisis_empresa.db')

# Ejecutar carga a SQL corregida
print("\n Cargando datos a SQL...")
cargar_a_sql_agrupado(dataframes, engine)


 Cargando datos a SQL...
 Tabla 'ventas_online' creada con 1000 registros
 Agregados 1000 registros a tabla: ventas_online
 Tabla 'inventario_almacen' creada con 1000 registros
 Tabla 'ventas_tienda' creada con 1000 registros
 Agregados 1000 registros a tabla: inventario_almacen
 Tabla 'inventario_bodega' creada con 1000 registros
 Agregados 1000 registros a tabla: inventario_bodega
 Agregados 1000 registros a tabla: ventas_tienda
ventas_online: 2000 registros
inventario_almacen: 2000 registros
ventas_tienda: 2000 registros
inventario_bodega: 2000 registros
Total general : 8000 registros
Total de tablas : 4 tablas


In [12]:
# Verificar la estructura de la base de datos
def explorar_base_datos(engine):
    from sqlalchemy import inspect

    inspector = inspect(engine)
    tablas = inspector.get_table_names()

    print(" TABLAS EN LA BASE DE DATOS:")
    for tabla in tablas:
        print(f"\n Tabla: {tabla}")
        columnas = inspector.get_columns(tabla)
        for columna in columnas:
            print(f"   ├─ {columna['name']} ({columna['type']})")

        # Mostrar 2 filas de ejemplo
        try:
            muestra = pd.read_sql(f"SELECT * FROM {tabla} LIMIT 2", engine)
            print(f"   └─ Muestra: {len(muestra)} filas")
        except Exception as e:
            print(f"   └─ Error: {e}")

# Ejecutar exploración
explorar_base_datos(engine)

 TABLAS EN LA BASE DE DATOS:

 Tabla: inventario_almacen
   ├─ sku_almacen (TEXT)
   ├─ producto_almacen (TEXT)
   ├─ categoria_almacen (TEXT)
   ├─ stock_actual_almacen (BIGINT)
   ├─ stock_minimo_almacen (BIGINT)
   ├─ proveedor_almacen (TEXT)
   ├─ costo_unitario_almacen (TEXT)
   ├─ ubicacion_almacen (TEXT)
   ├─ fuente_archivo (TEXT)
   └─ Muestra: 2 filas

 Tabla: inventario_bodega
   ├─ sku_bodega (TEXT)
   ├─ producto_bodega (TEXT)
   ├─ categoria_bodega (TEXT)
   ├─ stock_actual_bodega (BIGINT)
   ├─ stock_minimo_bodega (BIGINT)
   ├─ proveedor_bodega (TEXT)
   ├─ costo_unitario_bodega (TEXT)
   ├─ ubicacion_bodega (TEXT)
   ├─ fuente_archivo (TEXT)
   └─ Muestra: 2 filas

 Tabla: ventas_online
   ├─ orden_id (TEXT)
   ├─ cliente_id (TEXT)
   ├─ producto_online (TEXT)
   ├─ cantidad_online (BIGINT)
   ├─ precio_unitario_online (TEXT)
   ├─ fecha_compra_online (DATETIME)
   ├─ canal_online (TEXT)
   ├─ rating_online (BIGINT)
   ├─ fuente_archivo (TEXT)
   └─ Muestra: 2 filas

 

In [ ]:
# Sin SQL: Tendrías que hacer esto para cada análisis
df_ventas_enero = dataframes['ventas_online_enero']
df_ventas_febrero = dataframes['ventas_online_febrero']
df_ventas_marzo = dataframes['ventas_tienda_marzo']
# ... y manualmente unirlos, limpiarlos, etc.

# Con SQL: Todo está consolidado automáticamente
consulta_consolidada = "SELECT * FROM ventas_online"
df_todas_ventas_online = pd.read_sql(consulta_consolidada, engine)
print(f" Todas las ventas online: {len(df_todas_ventas_online)} registros")

# PRIMER TIPO DE CONSULTA: análisis de canal por ingresos
# ¿Cuál canal de ventas online genera más ingresos?
consulta_analisis_canal = """
SELECT
    canal_online,
    COUNT(*) as total_ventas,
    SUM(cantidad_online * precio_unitario_online) as ingresos_totales,
    AVG(cantidad_online * precio_unitario_online) as ticket_promedio
FROM ventas_online
GROUP BY canal_online
ORDER BY ingresos_totales DESC
"""

df_analisis_canal = pd.read_sql(consulta_analisis_canal, engine)
print(" Análisis por canal de ventas:")
print(df_analisis_canal)


# Comparar ventas online vs tienda física
consulta_comparativa = """
SELECT
    'Online' as tipo_venta,
    COUNT(*) as total_transacciones,
    SUM(cantidad_online * precio_unitario_online) as ingresos_totales,
    AVG(cantidad_online * precio_unitario_online) as ticket_promedio
FROM ventas_online
UNION ALL
SELECT
    'Tienda' as tipo_venta,
    COUNT(*) as total_transacciones,
    SUM(cantidad_tienda * precio_unitario_tienda) as ingresos_totales,
    AVG(cantidad_tienda * precio_unitario_tienda) as ticket_promedio
FROM ventas_tienda
"""

df_comparativo = pd.read_sql(consulta_comparativa, engine)
print(" Comparativa Online vs Tienda:")
print(df_comparativo)

#TIPO 3: Análisis Temporal
# Ventas por mes (usando las fechas de tus datos)
consulta_temporal = """
SELECT
    strftime('%Y-%m', fecha_compra_online) as mes,
    COUNT(*) as ventas_mensuales,
    SUM(cantidad_online * precio_unitario_online) as ingresos_mensuales
FROM ventas_online
GROUP BY mes
ORDER BY mes
"""

df_temporal = pd.read_sql(consulta_temporal, engine)
print(" Tendencia mensual de ventas:")
print(df_temporal)

# TIPO 4: agrupar por categorias
# Para inventario CORREGIDO

consulta_inventario = """
SELECT
    categoria_almacen as categoria,
    COUNT(*) as total_productos,
    SUM(stock_actual_almacen) as stock_total,
    AVG(costo_unitario_almacen) as costo_promedio,
    SUM(stock_actual_almacen * costo_unitario_almacen) as valor_inventario
FROM inventario_almacen
GROUP BY categoria_almacen
ORDER BY valor_inventario DESC
"""

df_inventario = pd.read_sql(consulta_inventario, engine)
print("\n Análisis de inventario por categoría:")
print(df_inventario)

# Esto SÍ muestra el poder de SQL - unir hojas similares automáticamente
consulta_unificada_ventas = """
SELECT
    'Online' as tipo_venta,
    producto_online as producto,
    COUNT(*) as total_ventas,
    SUM(cantidad_online * precio_unitario_online) as ingresos
FROM ventas_online
GROUP BY producto_online

UNION ALL

SELECT
    'Tienda' as tipo_venta,
    producto_tienda as producto,
    COUNT(*) as total_ventas,
    SUM(cantidad_tienda * precio_unitario_tienda) as ingresos
FROM ventas_tienda
GROUP BY producto_tienda

ORDER BY ingresos DESC
LIMIT 15
"""

df_unificado = pd.read_sql(consulta_unificada_ventas, engine)
print("\n Productos más vendidos - Online vs Tienda:")
print(df_unificado)

In [16]:
consulta_top_online = """
SELECT
    canal_online as canal,
    producto_online as producto,
    SUM(cantidad_online) as total_vendido,
    SUM(cantidad_online * precio_unitario_online) as ingresos_totales
FROM ventas_online
GROUP BY canal_online, producto_online
ORDER BY canal_online, total_vendido DESC
LIMIT 3"""

consulta_top_tienda = """
SELECT
    sucursal_tienda as canal,
    producto_tienda as producto,
    SUM(cantidad_tienda) as total_vendido,
    SUM(cantidad_tienda * precio_unitario_tienda) as ingresos_totales
FROM ventas_tienda
GROUP BY sucursal_tienda, producto_tienda
ORDER BY sucursal_tienda, total_vendido DESC
LIMIT 3"""

df_top_online = pd.read_sql(consulta_top_online, engine)
for i, (_, fila) in enumerate(df_top_online.iterrows(), 1):
    print(f"    {i}. {fila["producto"]} - {fila["total_vendido"]} unidades - ${fila["ingresos_totales"]:,.2f}")
df_top_tienda = pd.read_sql(consulta_top_tienda, engine)
for i, (_, fila) in enumerate(df_top_tienda.iterrows(), 1):
    print(f"    {i}. {fila["producto"]} - {fila["total_vendido"]} unidades - ${fila["ingresos_totales"]:,.2f}")
#lo mismo para top tienda

    1. Tablet - 353 unidades - $349,117.37
    2. Smartphone - 329 unidades - $326,739.16
    3. Auriculares - 315 unidades - $321,044.27
    1. Lavadora - 131 unidades - $201,196.68
    2. Microondas - 128 unidades - $217,325.28
    3. TV - 115 unidades - $182,233.65
